In [135]:
# import hail
import hail as hl

# 1. Set Up Dataset 

In [136]:
# read-in the HGDP+1kG dataset - after filtering, LD pruning and pca outliers removal 
mt = hl.read_matrix_table('gs://imary116/whole_dataset.mt')

In [137]:
# read-in regions bed file 
bed_file = hl.import_bed('gs://schema2/data/variant_qc/3.1_hl_filter-genotypes-schema-gnomad/0.2.1/grch38.gencode.v29.p8.merged.merged_by_exonid.cds.protein_coding.bed', reference_genome='GRCh38')


2022-09-19 16:45:57 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
  Loading field 'f3' as type str (user-supplied)
  Loading field 'f4' as type str (user-supplied)
  Loading field 'f5' as type str (not specified)


In [138]:
# filter mt to bed file regions only
mt_filt = mt.filter_rows(hl.is_defined(bed_file[mt.locus]))

In [139]:
# read in the schema exome mt 
mt_schema = hl.read_matrix_table('gs://schema2/data/variant_qc/3.1_hl_filter-genotypes-schema-gnomad/0.2.1/schema2-gnomad.common.mt')

# key the schema mt by locus only so the filtering can work
mt_schema = mt_schema.key_rows_by(mt_schema.locus)

In [140]:
# filter filtered mt to schema shared variants 
mt_filt2 = mt_filt.filter_rows(hl.is_defined(mt_schema.rows()[mt_filt.locus]))

In [141]:
mt_schema.count()

(7318, 346787)

In [142]:
mt_filt2.count()

2022-09-19 16:47:04 Hail: INFO: Ordering unsorted dataset with network shuffle


(325, 4097)

In [143]:
## In order to combine two datasets, three requirements must be met:
# 1. The row keys must match
# 2. The column key schemas and column schemas must match.
# 3. The entry schemas must match.

# filtered ds 
mt_filt2_unkeyed = mt_filt2.key_cols_by().key_rows_by() # no key mt
mt_filt2_clean = mt_filt2_unkeyed.select_cols(mt_filt2_unkeyed.s) # select the samples
mt_filt2_clean = mt_filt2_clean.select_rows(mt_filt2_clean.locus, mt_filt2_clean.alleles) # select locus and alleles 
mt_filt2_clean = mt_filt2_clean.select_entries(mt_filt2_clean.GT) # select GT 
mt_filt2_clean = mt_filt2_clean.key_cols_by('s').key_rows_by(*['locus', 'alleles']) # put back the keys


# schema ds 
mt_schema_unkeyed = mt_schema.key_cols_by().key_rows_by() # no key mt
mt_schema_clean = mt_schema_unkeyed.select_cols(mt_schema_unkeyed.s) # select the samples
mt_schema_clean = mt_schema_clean.select_rows(mt_schema_clean.locus, mt_schema_clean.alleles) # select locus and alleles 
mt_schema_clean = mt_schema_clean.select_entries(mt_schema_clean.GT) # select GT 
mt_schema_clean = mt_schema_clean.key_cols_by('s').key_rows_by(*['locus', 'alleles']) # put back the keys


# merge the two ds 
final_mt = mt_filt2_clean.union_cols(mt_schema_clean, row_join_type='outer') # include all rows from both mt 

In [144]:
final_mt.count() 

2022-09-19 16:47:42 Hail: INFO: Ordering unsorted dataset with network shuffle


(7325, 350884)

# 2. Run PCA

In [ ]:
# LD pruning - remove correlated variants 
pruned = hl.ld_prune(final_mt.GT, r2=0.1, bp_window_size=500000) #   
final_mt_pruned = final_mt.filter_rows(hl.is_defined(pruned[final_mt.row_key])) 

2022-09-19 16:52:13 Hail: INFO: ld_prune: running local pruning stage with max queue size of 765 variants
2022-09-19 16:52:17 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-09-19 16:53:30 Hail: INFO: wrote table with 5491 rows in 14901 partitions to /tmp/WR9MtJg9gckVIG5zi2zDgb
    Total size: 709.11 KiB
    * Rows: 709.10 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  6 rows (316.00 B)
2022-09-19 16:53:34 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-09-19 17:05:51 Hail: INFO: Wrote all 172 blocks of 5491 x 350884 matrix with block size 4096.
2022-09-19 18:24:42 Hail: INFO: Wrote all 172 blocks of 5491 x 350884 matrix with block size 4096.


In [ ]:
# separate related and unrelated individuals 

# compute kinship statistic
relatedness_ht = hl.pc_relate(final_mt_pruned.GT, min_individual_maf=0.05, min_kinship=0.05, statistics='kin', k=20).key_by() 

# identify closely related individuals in pairs (list of sample IDs) 
related_samples_to_remove = hl.maximal_independent_set(relatedness_ht.i, relatedness_ht.j, False) 

# subset filtered mt to samples that are NOT present in the list of related individuals  
mt_unrel = final_mt_pruned.filter_cols(hl.is_defined(related_samples_to_remove[final_mt_pruned.col_key]), keep=False) 

# do the same as above but this time subset to samples that are present in the related-individuals list   
mt_rel = final_mt_pruned.filter_cols(hl.is_defined(related_samples_to_remove[final_mt_pruned.col_key]), keep=True) 

In [ ]:
# PCA and projection functions
def run_pca(mt: hl.MatrixTable, reg_name:str, out_path: str, overwrite: bool = False):
    """
    Runs PCA on a data set
    :param mt: data set to run PCA on
    :param reg_name: region name for saving output purposes
    :param out_path: path for where to save the outputs
    :return:
    """

    pca_evals, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, k=20, compute_loadings=True)
    pca_mt = mt.annotate_rows(pca_af=hl.agg.mean(mt.GT.n_alt_alleles()) / 2)
    pca_loadings = pca_loadings.annotate(pca_af=pca_mt.rows()[pca_loadings.key].pca_af)
    pca_scores = pca_scores.transmute(**{f'PC{i}': pca_scores.scores[i - 1] for i in range(1, 21)})
    
    pca_scores.export(out_path + reg_name + '_scores.txt.bgz')  # save individual-level-genetic-region PCs
    pca_loadings.write(out_path + reg_name + '_loadings.ht', overwrite)  # save PCA loadings

    
from gnomad.sample_qc.ancestry import *

def project_individuals(pca_loadings, project_mt, reg_name:str, out_path: str, overwrite: bool = False):
    """
    Project samples into predefined PCA space
    :param pca_loadings: existing PCA space of unrelated samples 
    :param project_mt: matrix table of related samples to project  
    :param reg_name: region name for saving output purposes
    :param out_path: path for where to save PCA projection outputs
    :return:
    """
    ht_projections = pc_project(project_mt, pca_loadings)  
    ht_projections = ht_projections.transmute(**{f'PC{i}': ht_projections.scores[i - 1] for i in range(1, 21)}) 
    ht_projections.export(out_path + reg_name + '_projected_scores.txt.bgz') # save output

In [ ]:
# run PCA on the unrelated samples
run_pca(mt_unrel, 'global', 'gs://imary116/pca/', False)  

# read in the PCA loadings of the unrelated samples
loadings = hl.read_table('gs://imary116/pca/global_loadings.ht') 

# project the related samples onto the unrelated-samples' PC space 
project_individuals(loadings, mt_rel, 'global', 'gs://imary116/pca/', False) 